In [6]:
import pandas as pd
import dtale
import plotly.express as px 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [20]:
df = pd.read_csv("../data/outcomes.csv")

df.head()

,Unnamed: 0,CallerNum,Treatment Plan - Is the treatment plan completed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Birth_Year,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Treatment Plan - Was a treatment plan developed?
0,0,3638169,NaN,NaN,1960,Yes,Yes,NaN
1,1,4431194,NaN,NaN,1959,Yes,Yes,NaN
2,2,2941341,No,No,1942,Yes,No,Yes
3,3,3823933,No,No,1960,Yes,Yes,Yes
4,4,3348256,No,No,1955,Yes,Yes,Yes


In [22]:
#print(df.nunique())

In [59]:
sorted = df.sort_values(['CallerNum',
                         'Treatment Plan - Is the treatment plan completed?',
                         'Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)',
                         'Base-line Oral and Conditions Questions - Are you limited in what you can eat?']).drop_duplicates(['CallerNum'], keep='first')
sorted.shape

(10711, 8)

In [47]:
# DISCLAIMER: 'df' refers to the data you passed in when calling 'dtale.show'

duplicates = sorted.duplicated(['CallerNum'], keep='first')
sorted= sorted[~duplicates]
if isinstance(sorted, (pd.DatetimeIndex, pd.MultiIndex)):
    sorted = sorted.to_frame(index=False)

# remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
sorted = sorted.reset_index().drop('index', axis=1, errors='ignore')
sorted.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers

sorted.head()

,Unnamed: 0,CallerNum,Treatment Plan - Is the treatment plan completed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Birth_Year,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Treatment Plan - Was a treatment plan developed?
0,47543,-2,NaN,NaN,1953,No,No,Yes
1,609,-1,No,No,1900,NaN,NaN,Yes
2,34169,2384106,No,No,1944,Yes,Yes,Yes
3,3701,2384744,No,No,1954,Yes,No,Yes
4,54215,2384833,Yes,Yes,1942,Yes,No,Yes


In [48]:
print(sorted['CallerNum'].value_counts())

4149518    1
2415278    1
2854192    1
2504441    1
3388470    1
          ..
4357493    1
2469239    1
3102072    1
2776441    1
2525183    1
Name: CallerNum, Length: 10711, dtype: int64


In [56]:
completed_both = sorted[(sorted['Treatment Plan - Is the treatment plan completed?'] == "Yes") & (sorted['Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'] == 'Yes')]
completed_both.tail(50)

,Unnamed: 0,CallerNum,Treatment Plan - Is the treatment plan completed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Birth_Year,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Treatment Plan - Was a treatment plan developed?
8679,5364,3952166,Yes,Yes,1948,Yes,NaN,Yes
8712,15279,3965464,Yes,Yes,1957,Yes,Yes,Yes
8728,14767,3970415,Yes,Yes,1960,Yes,Yes,No
8791,14615,3999952,Yes,Yes,1949,Yes,NaN,Yes
8847,14045,4020568,Yes,Yes,1957,Yes,Yes,Yes
8906,8696,4043526,Yes,Yes,1948,Yes,NaN,Yes
8962,7446,4062947,Yes,Yes,1960,Yes,Yes,No
9022,12892,4088461,Yes,Yes,1955,Yes,No,Yes
9030,10805,4089248,Yes,Yes,1932,Yes,No,Yes
9070,5096,4104549,Yes,Yes,1961,Yes,No,Yes


In [57]:
print(completed_both['CallerNum'].value_counts())

2830336    1
2924853    1
2393764    1
2641164    1
2893859    1
          ..
3089764    1
2560356    1
3056404    1
2872678    1
2721448    1
Name: CallerNum, Length: 743, dtype: int64


In [58]:
d = dtale.show(completed_both)

#open it in a new window in browser

d.open_browser()